In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

import os
import base64
import zlib
import pickle
import sys
import os

sys.path.append("../")
from my_utils.sandbox_fusion import compute_score

/opt/anaconda/envs/diff-direction/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
MODEL_NAME = "bigcode/starcoder2-15b"
MODEL_ALIAS = "-".join(MODEL_NAME.split("/"))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cuda:0")

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
lcb_codegen = load_dataset("livecodebench/code_generation_lite", version_tag="release_v2")

In [ ]:
lcb_codegen

DatasetDict({
    test: Dataset({
        features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
        num_rows: 511
    })
})

In [ ]:
# compute_score(
#     sandbox_fusion_url, concurrent_semaphore, memory_limit_mb, completion, test_cases, continuous=False, timeout=10
# )

In [ ]:
lcb_codegen["test"][0]

{'question_title': 'A. Short Sort',
 'question_content': 'There are three cards with letters $\\texttt{a}$, $\\texttt{b}$, $\\texttt{c}$ placed in a row in some order. You can do the following operation at most once: \n\n \n-  Pick two cards, and swap them.  Is it possible that the row becomes $\\texttt{abc}$ after the operation? Output "YES" if it is possible, and "NO" otherwise.\n\nInput\n\nThe first line contains a single integer $t$ ($1 \\leq t \\leq 6$)\xa0— the number of test cases.\n\nThe only line of each test case contains a single string consisting of each of the three characters $\\texttt{a}$, $\\texttt{b}$, and $\\texttt{c}$ exactly once, representing the cards.\n\nOutput\n\nFor each test case, output "YES" if you can make the row $\\texttt{abc}$ with at most one operation, or "NO" otherwise.\n\nYou can output the answer in any case (for example, the strings "yEs", "yes", "Yes" and "YES" will be recognized as a positive answer).Sample Input 1:\n6\n\nabc\n\nacb\n\nbac\n\nbca

In [ ]:
def decode_private_tests(encoded_string):
    decoded_bytes = base64.b64decode(encoded_string)
    decompressed = zlib.decompress(decoded_bytes)
    decoded_test_case_dict = pickle.loads(decompressed)
    return decoded_test_case_dict

In [ ]:
# Get the first question content
question = lcb_codegen["test"][0]["question_content"]
starter_code = lcb_codegen["test"][0]["starter_code"]

# Format the prompt
prompt = f"{question}\n{starter_code}"

formatted_prompt = tokenizer.apply_chat_template([{"role": "system", "content": "You are "},
                                                  {"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)

# Tokenize and generate
inputs = tokenizer(formatted_prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=512, temperature=0.7, top_p=0.9)

# Decode the completion
completion = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
completion = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]: ], skip_special_tokens=True)[0]
print(completion)

To determine whether it's possible to rearrange the given sequence of three cards into the sequence `abc` using at most one swap, we can follow these steps for each test case:

1. **Check if the sequence is already `abc`**: If so, no swap is needed, and the answer is "YES".

2. **Check if exactly one swap is required**:
   - Identify the positions of `a`, `b`, and `c
